<a href="https://colab.research.google.com/github/agatino/tesi-transformer-italiano/blob/main/tesi_grillo_codice_pdfqa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PdfQA – Sistema di Question Answering per documenti Pdf (in italiano)
**6 giugno 2022**

##Architettura di riferimento 
* Haystack 
* Elasticsearch
* Transformers 

**Usare runtime = GPU**

### Altri tool:
* xpdf-tools-linux-4.04 (per conversione da pdf)


##Passi elaborativi

1. **Configurazione iniziale**: installazione ed import delle principali librerie software necessarie per eseguire l’applicazione 
2. **Preprocessing**: conversione e normalizzazione secondo un formato opportuno dei documenti che compongono il domain del sistema
3. **Retriever/Reader Pipeline**: questa sezione è il cuore dell’applicazione di Question Answering; qui sono configurati le componenti dell’architettura Information Retrieval QA secondo il framework Haystack adottato
4. **Question Answering**: formulazione delle domande ed elaborazione delle risposte

Il file pdf utilizzato è:

* **goal3.pdf** (Goal 3 – Salute e benessere) scaricato ddal sito ISTAT https://www.istat.it/storage/rapporti-tematici/sdgs/2021/goal3.pdf

Il modello Transformer utilizzato è:
* **mrm8488/bert-italian-finedtuned-squadv1-it-alfa** scaricato da https://huggingface.co/mrm8488/bert-italian-finedtuned-squadv1-it-alfa.





#GPU Colab Runtime

È opportuno utilizzare una GPU runtime di Colab
Dal menù in alto di Colab selezionare "Runtime" e "cambia tipo di runtime"


# 1.Configurazione iniziale: installazione delle librerie necessarie 



## 1.1 Haystack

In [ ]:
# Haystack
!pip install --upgrade pip
!pip install --quiet git+https://github.com/deepset-ai/haystack.git#egg=farm-haystack[colab,ocr]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.1 MB 9.6 MB/s 
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 60.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 77.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.8/17.8 MB 67.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.7/374.7 kB 31.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 58.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 5.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  P

## 1.2 Altre librerie
### 1.2.1 xpdf-tools-linux-4.04

Xpdf-tool è un toolkit di conversione dal formato pdf che include un estrattore di testo, un convertitore di immagini, un convertitore HTML e altro ancora

In [ ]:
# xpdf-tools-linux-4.04
!wget --no-check-certificate https://dl.xpdfreader.com/xpdf-tools-linux-4.04.tar.gz
!tar -xvf xpdf-tools-linux-4.04.tar.gz && sudo cp xpdf-tools-linux-4.04/bin64/pdftotext /usr/local/bin

--2022-06-07 08:39:05--  https://dl.xpdfreader.com/xpdf-tools-linux-4.04.tar.gz
Resolving dl.xpdfreader.com (dl.xpdfreader.com)... 45.79.72.155
Connecting to dl.xpdfreader.com (dl.xpdfreader.com)|45.79.72.155|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 23687259 (23M) [application/x-gzip]
Saving to: ‘xpdf-tools-linux-4.04.tar.gz’

xpdf-tools-linux-4. 100%[===================>]  22.59M  76.5MB/s    in 0.3s    

2022-06-07 08:39:06 (76.5 MB/s) - ‘xpdf-tools-linux-4.04.tar.gz’ saved [23687259/23687259]

xpdf-tools-linux-4.04/
xpdf-tools-linux-4.04/CHANGES
xpdf-tools-linux-4.04/COPYING3
xpdf-tools-linux-4.04/INSTALL
xpdf-tools-linux-4.04/ANNOUNCE
xpdf-tools-linux-4.04/bin64/
xpdf-tools-linux-4.04/bin64/pdftopng
xpdf-tools-linux-4.04/bin64/pdftohtml
xpdf-tools-linux-4.04/bin64/pdfinfo
xpdf-tools-linux-4.04/bin64/pdffonts
xpdf-tools-linux-4.04/bin64/pdfimages
xpdf-tools-linux-4.04/bin64/pdftotext
xpdf-tools-linux-4.04/bin64/pdftoppm
xpdf-tools-linux-4.04/bin64/pd

### 1.2.2 Haystack utils
Utility per la gestione di particolari funzioni della pipeline Haystack

In [ ]:
# Haystack utils
from haystack.nodes import TextConverter, PDFToTextConverter, DocxToTextConverter, PreProcessor
from haystack.utils import convert_files_to_docs, fetch_archive_from_http, clean_wiki_text, print_answers
from haystack.nodes import FARMReader, TransformersReader

INFO - haystack.modeling.model.optimization -  apex not found, won't use it. See https://nvidia.github.io/apex/
ERROR - root -  Failed to import 'magic' (from 'python-magic' and 'python-magic-bin' on Windows). FileTypeClassifier will not perform mimetype detection on extensionless files. Please make sure the necessary OS libraries are installed if you need this functionality.


# 2.Pre processing 


## 2.1 Classe Converter
Haystack fornisce una serie di classi di conversione – i converter - progettate per facilitare la trasformazione di archivi testuali in documenti che possono essere elaborati dalla pipeline Haystack. 
Esistono converter per file txt, pdf e docx. 
Il parametro valid_languages=["it"] non converte i file nella lingua di destinazione, ma controlla se la conversione ha funzionato come previsto.
Il parametro remove_numeric_tables=True elimina le tabelle contenenti soli valori numerici che non vengono quindi prese in considerazione.

In [ ]:
# Converter
converter = PDFToTextConverter(remove_numeric_tables=True, valid_languages=["it"])


INFO - haystack.telemetry -  Haystack sends anonymous usage data to understand the actual usage and steer dev efforts towards features that are most meaningful to users. You can opt-out at anytime by calling disable_telemetry() or by manually setting the environment variable HAYSTACK_TELEMETRY_ENABLED as described for different operating systems on the documentation page. More information at https://haystack.deepset.ai/guides/telemetry


In [ ]:
### Caricamento del documento pdf
from google.colab import files

uploaded = files.upload()

Saving goal3.pdf to goal3.pdf


In [ ]:
doc_pdf = converter.convert(file_path="/content/goal3.pdf") 

In [ ]:
doc_pdf

[<Document: {'content': "2. Analisi delle misure statistiche per Goal\n\n\nASSICURARE LA SALUTE E IL BENESSERE PER TUTTI E PER TUTTE LE ETÀ1\n\nIn sintesi\n• Per misurare l’impatto dell’epidemia di COVID-19 è utile considerare l’eccesso di mortalità, che tiene conto dei decessi attribuiti direttamente al COVID-19 e di quelli indirettamente collegati. Nel 2020 il totale dei decessi per il complesso delle cause è stato il più alto mai registrato in Italia dal secondo dopoguerra: 746.146 decessi; 100.526 decessi in più rispetto alla media 2015-2019 (con una percentuale di eccesso di mortalità del 15,6%).\n• La copertura vaccinale antinfluenzale della stagione invernale 2020/2021 è sensibilmente aumentata rispetto alla stagione precedente passando, per la popolazione anziana sopra i 65 anni, dal 54,6% al 66,5%.\n• Nel 2020, al netto della struttura per età della popolazione, la prevalenza di persone che dichiarano di avere il diabete è del 6,2%, in lieve aumento rispetto al 5,8% del 2010. 

## 2.2 Classe PreProcessor

La classe Haystack  PreProcessor è progettata per pulire il testo e dividere il testo in unità di dimensioni minori. 

La suddivisione del testo in più unita ha un impatto molto significativo sulle prestazioni del sistema in particolare se si utilizzano modelli Dense Passage Retrieval. 

In generale è consigliato dividere il testo in piccoli documenti di circa 100 parole per metodi di retrieval densi e non più di 10.000 parole per metodi sparsi.

La classe PreProcessor prende come input uno dei documenti creati dal converter, esegue vari passaggi di pulizia e li divide in più documenti più piccoli. 

* clean_empty_lines normalizza 3 o più righe vuote consecutive in modo che siano solo due righe vuote
* clean_whitespace rimuove qualsiasi spazio bianco all'inizio o alla fine di ogni riga nel testo
* clean_header_footer rimuove qualsiasi testo lungo di intestazione o piè di pagina che si ripete su ogni pagina
* split_by determina per quale unità è suddiviso il documento: 'word', 'sentence' o 'passage' 
* split_length imposta un numero massimo di unità di 'word', 'sentence' o 'passage' per documento di output
* split_respect_sentence_boundary assicura che i confini del documento non cadano nel mezzo delle frasi.

Per impostazione predefinita, il modello DensePassageRetriever di Haystack ha una lunghezza massima di 256 token. 

Pertanto è consigliato che i documenti contengano molto meno parole; le migliori prestazioni si hanno con documenti lunghi circa 100 parole.

In [ ]:
preprocessor = PreProcessor(
    clean_empty_lines=True,
    clean_whitespace=True,
    clean_header_footer=False,
    split_by="word",
    split_length=100,
    split_respect_sentence_boundary=True,
    split_overlap=10
)
preprocessed = preprocessor.process(doc_pdf)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


100%|██████████| 1/1 [00:00<00:00, 41.27docs/s]


In [ ]:
docs_default = preprocessor.process(doc_pdf)
print("n_docs_input: 1\nn_docs_output: {len(docs_default)}")

100%|██████████| 1/1 [00:00<00:00, 71.18docs/s]

n_docs_input: 1
n_docs_output: {len(docs_default)}


## 2.4 Window approach

Una strategia comunemente usata per dividere documenti lunghi, specialmente per le risposte alle domande, è il window approach o l'approccio della finestra scorrevole. 
Impostando i parametri 

*	split_length=10
*	split_overlap=3, 

i documenti hanno questo format:

*	doc1 = words[0:10]
*	doc2 = words[7:17]
*	doc3 = words[14:24]
*	...



In [ ]:
# Sliding window approach

preprocessor_sliding_window = PreProcessor(split_overlap=3, split_length=10, split_respect_sentence_boundary=False)
docs_sliding_window = preprocessor_sliding_window.process(doc_pdf)

doc1 = docs_sliding_window[0].content[:200]
doc2 = docs_sliding_window[1].content[:100]
doc3 = docs_sliding_window[2].content[:100]

print('Document 1: "' + doc1 + '..."')
print('Document 2: "' + doc2 + '..."')
print('Document 3: "' + doc3 + '..."')

100%|██████████| 1/1 [00:00<00:00, 128.46docs/s]

Document 1: "2. Analisi delle misure statistiche per Goal

ASSICURARE LA SALUTE E..."
Document 2: "LA SALUTE E IL BENESSERE PER TUTTI E PER TUTTE..."
Document 3: "E PER TUTTE LE ETÀ1

In sintesi
• Per misurare l’impatto dell’epidemia..."


## 2.5 Elasticsearch

Elasticsearch è un datastoreNoSQL - quindi non relazionale - progettato per la memorizzazione, ricerca e il recupero di documenti.
Rispetto a un database convenzionale è utilizzato in particolare per dati semi strutturati o non strutturati, soprattutto documenti basati su testo.
La sua capacità di archiviare enormi volumi di dati e di filtrarli rapidamente con funzioni di ricerca full-text lo rendono particolarmente adatto per lo sviluppo di sistemi di QA. 

Per installare Elasticsearch in Colab occorre prima scaricare il programma attraverso il comando **wget**, poi decomprimerlo con **tar** e infine mandarlo in esecuzione con **chown**.


In [ ]:
#Colab: start Elasticsearch from source

! wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.9.2-linux-x86_64.tar.gz -q
! tar -xzf elasticsearch-7.9.2-linux-x86_64.tar.gz
! chown -R daemon:daemon elasticsearch-7.9.2



**Popen()**

La funzione Popen() serve a generare generare un nuovo processo
Nella funzione Popen(), gli args specificano il programma che si vuole  eseguire, mentre stdout=PIPE crea una nuova pipe per l'output standard e stderr=STDOUT raccoglie gli errori nella stessa pipe

In [ ]:
import os
from subprocess import Popen, PIPE, STDOUT
es_server = Popen(['elasticsearch-7.9.2/bin/elasticsearch'],
                   stdout=PIPE, stderr=STDOUT,
                   preexec_fn=lambda: os.setuid(1)  # as daemon
                  )
# attendere 30 secondi che ElasticSearch sia partito 
! sleep 30

In [ ]:
# Carica i documenti in document_store
# delete_all_documents() serve a ripulire document_store prima di caricare i dati

from haystack.document_stores import ElasticsearchDocumentStore
document_store = ElasticsearchDocumentStore()
document_store.delete_all_documents()
document_store.write_documents(preprocessed)

WARNING - haystack.document_stores.elasticsearch -  DEPRECATION WARNINGS: 
                1. delete_all_documents() method is deprecated, please use delete_documents method
                For more details, please refer to the issue: https://github.com/deepset-ai/haystack/issues/1045
                


#3 Retriever/reader Pipeline


## 3.1 Retriever config
I retriever permettono di restringere l'ambito del testo che il reader deve analizzare per rispondere a una determinata domanda. 

Qui si usa l'algoritmo BM25 predefinito di Elasticsearch. 

In alternativa è possibile usare TfidfRetriever. 


In [ ]:
from haystack.nodes import DensePassageRetriever, FARMReader 
retriever = DensePassageRetriever(document_store=document_store) 

INFO - haystack.modeling.utils -  Using devices: CUDA:0
INFO - haystack.modeling.utils -  Number of GPUs: 1


Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/493 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/492 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DPRQuestionEncoderTokenizer'. 
The class this function is called from is 'DPRContextEncoderTokenizerFast'.
INFO - haystack.modeling.model.language_model -  LOADING MODEL
INFO - haystack.modeling.model.language_model -  =============
INFO - haystack.modeling.model.language_model -  Could not find facebook/dpr-ctx_encoder-single-nq-base locally.
INFO - haystack.modeling.model.language_model -  Looking on Transformers Model Hub (in local cache and online)...


Downloading:   0%|          | 0.00/418M [00:00<?, ?B/s]

INFO - haystack.modeling.model.language_model -  Loaded facebook/dpr-ctx_encoder-single-nq-base


## 3.2 Reader config

Il Reader scansiona in dettaglio i testi restituiti dai retriever ed estrae le k migliori risposte.
Haystack supporta reader basati sui framework FARM e Transformers e con entrambi si può caricare un modello locale o un modello presente su Hugging Face (https://huggingface.co/models). 
Qui si usa il modello di Question Answering **mrm8488/bert-italian-finedtuned-squadv1-it-alfa** scaricato da https://huggingface.co/mrm8488/bert-italian-finedtuned-squadv1-it-alfa.

Vedi: da https://haystack.deepset.ai/docs/latest/languagesmd 


Un Reader alternativo è TransformersReader che sfrutta la pipeline di Transformers.

In [ ]:
#from haystack.nodes import FARMReader
reader = FARMReader("mrm8488/bert-italian-finedtuned-squadv1-it-alfa")

#o in alternativa
#from haystack.nodes import TransformersReader
#reader = TransformersReader("mrm8488/bert-italian-finedtuned-squadv1-it-alfa")

INFO - haystack.modeling.utils -  Using devices: CUDA:0
INFO - haystack.modeling.utils -  Number of GPUs: 1
INFO - haystack.modeling.model.language_model -  LOADING MODEL
INFO - haystack.modeling.model.language_model -  =============
INFO - haystack.modeling.model.language_model -  Could not find mrm8488/bert-italian-finedtuned-squadv1-it-alfa locally.
INFO - haystack.modeling.model.language_model -  Looking on Transformers Model Hub (in local cache and online)...


Downloading:   0%|          | 0.00/442 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/419M [00:00<?, ?B/s]

INFO - haystack.modeling.model.language_model -  Loaded mrm8488/bert-italian-finedtuned-squadv1-it-alfa


Downloading:   0%|          | 0.00/40.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/230k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

INFO - haystack.modeling.utils -  Using devices: CUDA
INFO - haystack.modeling.utils -  Number of GPUs: 1
INFO - haystack.modeling.infer -  Got ya 2 parallel workers to do inference ...
INFO - haystack.modeling.infer -   0     0  
INFO - haystack.modeling.infer -  /w\   /w\ 
INFO - haystack.modeling.infer -  /'\   / \ 


### Dense vector embeddings

In [ ]:
document_store.update_embeddings(retriever)

INFO - haystack.document_stores.elasticsearch -  Updating embeddings for all 46 docs ...


Updating embeddings:   0%|          | 0/46 [00:00<?, ? Docs/s]

Create embeddings:   0%|          | 0/48 [00:00<?, ? Docs/s]

#4 Question Answering

Haystack  permette di riunire tutte le componenti viste in una unica pipeline.
Le pipeline sono grafici aciclici diretti (DAG) che si possono facilmente personalizzare per i vari casi d'uso. 
Per velocizzare le cose, Haystack include anche alcune pipeline predefinite. 
Uno di queste è l'ExtractiveQAPipeline che combina un retriever e un reader per rispondere alle domande.

In [ ]:
from haystack.pipelines import ExtractiveQAPipeline
pipeline = ExtractiveQAPipeline(reader, retriever)

## 4.1 Information extraction

Come prima prova si inserisce il testo di sei domande relative agli argomenti trattati nel pdf usato come domain

In [ ]:
questions = [ "Totale dei decessi nel 2020?",
             "Deceduti oltre 80 anni?",
             "Deceduti tra 65-79 anni?",
              "Individui maggiorenni con diabete?",
              "Speranza di vita?",
              "Speranza di vita per le donne?"
              ]


In [ ]:
answers = []
for question in questions:
    prediction = pipeline.run(query=question,
                              params = { "Retriever": {"top_k": 20},
                                         "Reader": { "top_k": 1 } })
    answers.append(prediction)


Inferencing Samples:   0%|          | 0/1 [00:00<?, ? Batches/s]/usr/local/lib/python3.7/dist-packages/haystack/modeling/model/prediction_head.py:483: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  start_indices = flat_sorted_indices // max_seq_len
Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00, 57.68 Batches/s]


In [ ]:
for answer in answers:
    print("Q:", answer['query'])
    print("A:", answer['answers'][0].answer)
    print("\n")

Q: Totale dei decessi nel 2020?
A: 746.146


Q: Deceduti oltre 80 anni?
A: 486.255


Q: Deceduti tra 65-79 anni?
A: 184.708


Q: Individui maggiorenni con diabete?
A: popolazione anziana sopra i 65 anni


Q: Speranza di vita?
A: 79,7 anni


Q: Speranza di vita per le donne?
A: 84,4 anni





## 4.2 Questioning


È possibile analizzare il dettaglio delle risposte fornite visualizzando il contesto (cioè il brano di testo usato come contesto) e la probabilità assegnata alla risposta individuate.
Per fare questa prova valorizziamo top_k = 3 cioè chiediamo di avere non la migliore risposta ma le tre migliori possibili risposte 

### prima domanda


In [ ]:
prediction = pipeline.run(query="Totale dei decessi nel 2020?", params={"Retriever": {"top_k": 20}, "Reader": {"top_k": 3}}
)

Inferencing Samples:   0%|          | 0/1 [00:00<?, ? Batches/s]/usr/local/lib/python3.7/dist-packages/haystack/modeling/model/prediction_head.py:483: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  start_indices = flat_sorted_indices // max_seq_len
Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00, 43.16 Batches/s]


In [ ]:
# visualizzazione delle previsioni
from pprint import pprint
pprint(prediction)

{'answers': [<Answer {'answer': '746.146', 'type': 'extractive', 'score': 0.9770419597625732, 'context': 'se è stato il più alto mai registrato in Italia dal secondo dopoguerra: 746.146 decessi; 100.526 decessi in più rispetto alla media 2015-2019 (con una', 'offsets_in_document': [{'start': 450, 'end': 457}], 'offsets_in_context': [{'start': 72, 'end': 79}], 'document_id': '70854c3c50f404beba18136b503043c7', 'meta': {'_split_id': 0}}>,
             <Answer {'answer': '746.146', 'type': 'extractive', 'score': 0.953663170337677, 'context': 'se è stato il più alto mai registrato in Italia dal secondo dopoguerra: 746.146 decessi; 100.526 decessi in più rispetto alla media 2015-2019 (con una', 'offsets_in_document': [{'start': 129, 'end': 136}], 'offsets_in_context': [{'start': 72, 'end': 79}], 'document_id': '50df016e16bfd58bc2dc1790e4fa6304', 'meta': {'_split_id': 1}}>,
             <Answer {'answer': '82', 'type': 'extractive', 'score': 0.9065957069396973, 'context': 'la mortalità ha av

In [ ]:
# è possibile selezionare il livello di informazioni ottenute con ogni risposta
# selezionando `minimum`, `medium` o `all`
print_answers(prediction, details="medium")


Query: Totale dei decessi nel 2020?
Answers:
[   {   'answer': '746.146',
        'context': 'se è stato il più alto mai registrato in Italia dal '
                   'secondo dopoguerra: 746.146 decessi; 100.526 decessi in '
                   'più rispetto alla media 2015-2019 (con una',
        'score': 0.9770419597625732},
    {   'answer': '746.146',
        'context': 'se è stato il più alto mai registrato in Italia dal '
                   'secondo dopoguerra: 746.146 decessi; 100.526 decessi in '
                   'più rispetto alla media 2015-2019 (con una',
        'score': 0.953663170337677},
    {   'answer': '82',
        'context': 'la mortalità ha avuto un effetto sulla speranza di vita '
                   'alla nascita che, nel 2020, è scesa a 82 anni, 1,2 anni '
                   'sotto il livello dell’anno precedente.',
        'score': 0.9065957069396973}]


### seconda domanda


In [ ]:
prediction = pipeline.run(query="Deceduti oltre 80 anni?", params={"Retriever": {"top_k": 20}, "Reader": {"top_k": 3}}
)

Inferencing Samples:   0%|          | 0/1 [00:00<?, ? Batches/s]/usr/local/lib/python3.7/dist-packages/haystack/modeling/model/prediction_head.py:483: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  start_indices = flat_sorted_indices // max_seq_len
Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00, 50.38 Batches/s]


In [ ]:
print_answers(prediction, details="medium")


Query: Deceduti oltre 80 anni?
Answers:
[   {   'answer': '486.255',
        'context': 'elle morti della popolazione con 80 anni e più: in totale '
                   'sono decedute 486.255 persone di 80 anni e oltre (76.708 '
                   'in più rispetto alla media del quin',
        'score': 0.9813368618488312},
    {   'answer': '486.255',
        'context': 'elle morti della popolazione con 80 anni e più: in totale '
                   'sono decedute 486.255 persone di 80 anni e oltre (76.708 '
                   'in più rispetto alla media del quin',
        'score': 0.9477532207965851},
    {   'answer': '82',
        'context': 'la mortalità ha avuto un effetto sulla speranza di vita '
                   'alla nascita che, nel 2020, è scesa a 82 anni, 1,2 anni '
                   'sotto il livello dell’anno precedente.',
        'score': 0.8618353009223938}]


### quarta domanda

In [ ]:
prediction = pipeline.run(query="Individui maggiorenni con diabete?", params={"Retriever": {"top_k": 20}, "Reader": {"top_k": 3}}
)

Inferencing Samples:   0%|          | 0/1 [00:00<?, ? Batches/s]/usr/local/lib/python3.7/dist-packages/haystack/modeling/model/prediction_head.py:483: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  start_indices = flat_sorted_indices // max_seq_len
Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00, 46.83 Batches/s]


In [ ]:
print_answers(prediction, details="medium")


Query: Individui maggiorenni con diabete?
Answers:
[   {   'answer': 'popolazione anziana sopra i 65 anni',
        'context': 'enda 2030 in Italia\n'
                   '\n'
                   'La copertura vaccinale rispetto alla popolazione anziana '
                   'sopra i 65 anni, popolazione target della strategia '
                   'antiinfluenzale, è a',
        'score': 0.8638966083526611},
    {   'answer': 'persone sopra i 65 anni',
        'context': 'uota scende al 42,5%; -1,2 p.p. ), e per età, a eccezione '
                   'delle persone sopra i 65 anni (che presentano però il '
                   'livello più alto di persone in eccesso',
        'score': 0.8243407607078552},
    {   'answer': '108.178',
        'context': 'In questo periodo si sono osservati 108.178 decessi in più '
                   'rispetto alla media dello stesso periodo degli anni '
                   '2015-2019, con una percentuale di ecces',
        'score': 0.7786733508110046}]


### quinta domanda

In [ ]:
prediction = pipeline.run(query="Speranza di vita?", params={"Retriever": {"top_k": 20}, "Reader": {"top_k": 3}}
)
print_answers(prediction, details="medium")

Inferencing Samples:   0%|          | 0/1 [00:00<?, ? Batches/s]/usr/local/lib/python3.7/dist-packages/haystack/modeling/model/prediction_head.py:483: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  start_indices = flat_sorted_indices // max_seq_len
Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00, 50.85 Batches/s]


Query: Speranza di vita?
Answers:
[   {   'answer': '79,7 anni',
        'context': 'La speranza di vita alla nascita degli uomini è scesa a '
                   '79,7 anni, ossia 1,4 anni in meno dell’anno precedente, '
                   'mentre per le donne si è attestata a 8',
        'score': 0.8156921565532684},
    {   'answer': 'ipertensione',
        'context': ' di 65-74 anni e il 55% degli ultra 75enni dichiarano di '
                   'soffrire di ipertensione. Tra le donne la quota di '
                   'ipertese è più bassa tra i 60-64 anni (29,',
        'score': 0.613576203584671},
    {   'answer': '2020',
        'context': 'n aumento del 9% del tasso di mortalità standardizzato '
                   'riferito all’anno 2020 rispetto a quello medio del periodo '
                   '2015-2019; L’aumento della mortalità',
        'score': 0.6135525405406952}]


### sesta domanda

In [ ]:
prediction = pipeline.run(query="Speranza di vita per le donne?", params={"Retriever": {"top_k": 20}, "Reader": {"top_k": 3}}
)
print_answers(prediction, details="medium")

Inferencing Samples:   0%|          | 0/1 [00:00<?, ? Batches/s]/usr/local/lib/python3.7/dist-packages/haystack/modeling/model/prediction_head.py:483: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  start_indices = flat_sorted_indices // max_seq_len
Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00, 53.64 Batches/s]


Query: Speranza di vita per le donne?
Answers:
[   {   'answer': '84,4 anni',
        'context': 'nni in meno dell’anno precedente, mentre per le donne si è '
                   'attestata a 84,4 anni, un anno di sopravvivenza in meno. '
                   'In aumento le patologie legate all',
        'score': 0.9849217534065247},
    {   'answer': '84,4 anni',
        'context': 'ni è scesa a 79,7 anni, ossia 1,4 anni in meno dell’anno '
                   'precedente, mentre per le donne si è attestata a 84,4 '
                   'anni, un anno di sopravvivenza in meno.',
        'score': 0.9449624717235565},
    {   'answer': '21%',
        'context': 'abete è più diffuso tra gli ultrasettantacinquenni, sia '
                   'uomini sia donne (21%; Figura 3.2). 3 \x07Si veda anche '
                   'Automobile Club d’Italia - ACI, Istituto ',
        'score': 0.8799931108951569}]


## 4.3 Answer analysis

In [ ]:
while(1):
    question = input("Inserire la domanda (quit per uscire): ") 
    if question == 'quit':
        break
    prediction = pipeline.run(query=question, params={"Retriever": {"top_k": 10}, "Reader": {"top_k": 1}})
    print_answers(prediction, details="medium")

Inserire la domanda (quit per uscire): persone in eccesso di peso?


Inferencing Samples:   0%|          | 0/1 [00:00<?, ? Batches/s]/usr/local/lib/python3.7/dist-packages/haystack/modeling/model/prediction_head.py:483: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  start_indices = flat_sorted_indices // max_seq_len
Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00, 37.18 Batches/s]



Query: persone in eccesso di peso?
Answers:
[   {   'answer': '56,1%',
        'context': 'studio basso (ovvero che hanno al massimo la licenza media '
                   'inferiore) il 56,1% è in eccesso di peso, rispetto al '
                   '34,4% di chi possiede un titolo di st',
        'score': 0.9527110755443573}]
Inserire la domanda (quit per uscire): persone in eccesso di peso con titolo di studio basso?


Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00, 45.10 Batches/s]



Query: persone in eccesso di peso con titolo di studio basso?
Answers:
[   {   'answer': '56,1%',
        'context': 'studio basso (ovvero che hanno al massimo la licenza media '
                   'inferiore) il 56,1% è in eccesso di peso, rispetto al '
                   '34,4% di chi possiede un titolo di st',
        'score': 0.9712694585323334}]
Inserire la domanda (quit per uscire): livello di incidenza di sindromi simil-influenzali


...


Query: Quanti dichiarano di essere affetti da ipertensione?
Answers:
[   {   'answer': '55%',
        'context': 'essivamente: il 34,6% degli uomini di 60-64 anni, il 45,3% '
                   'di quelli di 65-74 anni e il 55% degli ultra 75enni '
                   'dichiarano di soffrire di ipertensione.',
        'score': 0.981115996837616}]
Inserire la domanda (quit per uscire): Morti in indienti stradali?


Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00, 40.92 Batches/s]



Query: Morti in indienti stradali?
Answers:
[   {   'answer': 'ipertensione',
        'context': ' di 65-74 anni e il 55% degli ultra 75enni dichiarano di '
                   'soffrire di ipertensione. Tra le donne la quota di '
                   'ipertese è più bassa tra i 60-64 anni (29,',
        'score': 0.9362150728702545}]
Inserire la domanda (quit per uscire): Morti in incidenti stradali?


Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00, 30.51 Batches/s]



Query: Morti in incidenti stradali?
Answers:
[   {   'answer': '123.061',
        'context': 'i con lesioni a persone (90.821, pari a -29,5%), del '
                   'numero dei feriti (123.061; -32,0%) e del totale delle '
                   'vittime entro il trentesimo giorno dall’in',
        'score': 0.9922448694705963}]
Inserire la domanda (quit per uscire): quit
